In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from keras.preprocessing.sequence import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
import time

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
Using TensorFlow backend.


In [2]:
#vars
vocab_size = 5000
embedding_size = 128
max_words = 50

LSTM_units = 128
Dense_units = 1

start_data = time.time()

#data
data = pd.read_csv('train.csv', encoding='latin-1')

#initialize tokenizer
t = Tokenizer(num_words = vocab_size, split = ' ')
t.fit_on_texts(data['SentimentText'])

#divide into test and training
X_train = data[:int(data.shape[0]*0.8)].SentimentText
X_test = data[int(data.shape[0]*.8):].SentimentText

Y_train = data[:int(data.shape[0]*0.8)].Sentiment
Y_test = data[int(data.shape[0]*.8):].Sentiment

#convert string to int
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)

#make every tweet the same length
X_train = sequence.pad_sequences(X_train, max_words)
X_test = sequence.pad_sequences(X_test, max_words)

end_data = time.time()
print('======================================')
print('Time to manage data: ' + str(end_data - start_data))
print('======================================')

Time to manage data: 4.648736953735352


In [3]:
model = Sequential()

model.add(Embedding(vocab_size, embedding_size, input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(LSTM_units, dropout = .2, recurrent_dropout = .2))
model.add(Dense(Dense_units, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, Y_train, batch_size = 64, epochs = 1, validation_split = 0.20, shuffle = True, verbose = 1)

Train on 63992 samples, validate on 15999 samples
Epoch 1/1
63992/63992 [==============================] - 88s 1ms/step - loss: 7.1767 - acc: 0.5498 - val_loss: 6.5448 - val_acc: 0.5895
